Geotypes adds some high-level wrappers around geopandas and shapely to allow for easier use. They allow easy conversion between georeference schemes, better tracking of what coordinate system the variables are in and easy conversion between types.
These include:

* GeoPoint
* LL
* GeoPath
* GeoPoints
* GeoArea
* XY
* LocalPath
* LocalPoints
* LocalArea





In [ ]:
import geotypes

## GeoPoint
The GeoPoint is an x,y coordinate with an associated Coordinate Reference System (CRS). It allows easy conversion to LL and XY data types.

In [ ]:
p = geotypes.geometry.GeoPoint(x=151.28826000, y=-33.79798000, crs="epsg:4326") # initialise a lat, lon centred in Manly.
print("GeoPoint", p)
ll = p.to_ll() # Converts to lat,lon data type
print("In LL", ll)
p_utm = p.to_crs("EPSG:32756")  # converts to utm zone 56S
print("In UTM Zone 56S", p_utm)

## LL
The lat,lon data type is there for convience and to make it explicit about what is latitude, what is longitude.


In [ ]:
ll = geotypes.geometry.LL(-33.79798000,151.28826000)
print(ll)
# Convert to geopoint
p = ll.to_geopoint()
print(p)

### GeoPath
A path in geographic space, effectively a geopandas geoseries / shapely linestring with an associated CRS. It is commonly used to represent sampling transects (such as AUV transects). It has some convient functions to collect samples at set distances along the path - which can be done no-matter what 

In [ ]:
# Define a set of points. This is a simple broad grid in CRS "EPSG:32756"
points = [(342041.99358729343, 6259392.849573873),
 (342495.27623430215, 6259603.771833958),
 (342579.64503957686, 6259422.458960008),
 (342126.36269032373, 6259211.536449324),
 (342210.73189359176, 6259030.223443877),
 (342664.01394509675, 6259241.146205157)]

# create the geopath object
geopath = geotypes.geometry.GeoPath(points, crs="EPSG:32756")
print(geopath)

# Iterating over it returns a geopoint.
print("Geopoints:")
for p in geopath:
    print(p)
print("\n")

# Easily convert to other crs
geopath4326 = geopath.to_crs("EPSG:4326")
print(geopath4326)

# Sample points along the geopath
geopath_subsampled = geopath.sample_points_along_path(d=20) # sample a point every 20 meters.
print("Number of points in original {}, in subsampled {}".format(len(geopath), len(geopath_subsampled)))


### GeoArea

A GeoArea is a wrapper around a geopandas GeoSeries with just one element, a polygon or multipolygon. They are used within the situ framework to represent sampling bounds or focal areas. As such, they have easy access to random sampling.



In [ ]:
# Define a geoarea
geoarea = geotypes.geometry.GeoArea([(151.28412747953516, -33.78781691040293),
 (151.30572252046485, -33.78781691040293),
 (151.30572478467255, -33.805848131225765),
 (151.28412521532746, -33.805848131225765),
 (151.28412747953516, -33.78781691040293)], crs='EPSG:4326')

# get some samples from within it - these are GeoPoints:
for n in range(5):
    print(geoarea.random_point_geo())
# you can get also get points directly as LLs, regardless of the CRS.
for n in range(5):
    print(geoarea.random_point_ll())

In [ ]:
# Plot the overall area on the map
import folium
# Plot the area
m = geoarea.explore(style_kwds={'fill': False, 'color':'green'})
# Plot the geopath
geopath.explore(m=m)
# Plot the samples along the geopath
geotypes.geometry.GeoPoints(geopath.sample_points_along_path(50).to_geopoints()).explore(m=m)
# Plot some random samples from the geoarea
geotypes.geometry.GeoPoints([geoarea.random_point_ll() for _ in range(10)]).explore(m=m, style_kwds={'color':'red'})
folium.LayerControl().add_to(m)
display(m)



## Local Equivalents
There are local equivalents to the classes presented above. These include:
- XY: Equivalent of GeoPoint
- LocalPoints: Equivalent of GeoPoints
- LocalPath: Equivalent of GeoPath
- LocalArea: Equivalent of GeoArea
These are just wrappers around the shapely geometry classes, but offer some extra functionality, including being able to convert to and from the Geo equivalents.

In [ ]:
origin_ll = geoarea.centroid_ll()
# Convert the position to local
xy = p.to_local(origin_ll)
# Now go back to GeoPoint, this time with a different CRS.
pll = xy.to_geo(origin_ll, "epsg:4326")
# Print all to compare
print(p, xy, pll)


# Convert a GeoPath to a LocalPath
localpath = geopath.to_local(origin_ll)
# And go back again
geopath2 = localpath.to_geo(origin_ll, geopath.crs)